In [1]:
# Test the several architecture of ProtoNN versus the K-NN algorithm

# import ProtoNN code
import protoNN as protograph
import protoNNTrainer as prototrainer
import utils as utils
# import general modules
import numpy as np
import torch
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import gc

# need to change the dataset location
X, Y = utils.handleMNIST('../../../data_set/mnist.csv')

dataset = utils.CustomDatasetProtoNN(X, Y, rescaling=True, binary=False)

C:\Users\pietr\anaconda3\envs\edgeml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset input rescaled
Dataset loaded, input space: (60000, 784) , output space: (60000,)
Dataset reshaped, input space: (60000, 784) , output space: (60000, 10)


In [2]:
# Stratified k fold
k_folds = 5 # default 5 or 10
stratifiedk_fold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=1)
knn = KNeighborsClassifier(n_neighbors = 13)
print("Stratified KFOLD INITIALIZED")

Stratified KFOLD INITIALIZED


In [3]:
# fixed ProtoNN HYPERPARAMETERS
# regularizer of W, B and Z: HYPERPARAMETER
regW, regB, regZ = 0, 0, 0

# sparsity of W, B and Z: HYPERPARAMETER, CONSTRAINTS OF MEMORY
sparcityW, sparcityB, sparcityZ = 1, 1, 1

# initial learning rate of the OPTIMIZER: HYPERPARAMETER
learningRate = 0.1
# lossType: 'l2'
lossType= 'l2'

# TRAINING PHASE
# batchsize: HAS TO BE DEFINED
batchsize = 100
# epochs: HAS TO BE DEFINED
epochs = 100
# printStep: HAS TO BE DEFINED
printStep = 100
# valStep: HAS TO BE DEFINED
valStep = 100

# print virables of ProtoNN training
print("\nbatch size:\t", batchsize,
      "\nepochs:\t\t", epochs,
      "\nprint step:\t", printStep,
      "\nevaluation step:", valStep)


batch size:	 100 
epochs:		 100 
print step:	 100 
evaluation step: 100


In [4]:
# FOR 2 KB
size = [2]
size_parameters = [[3, 4], [3, 5], [2, 46]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.2
        if (index_size == 1):
            sparcityZ = 0.1
        if (index_size == 2):
            sparcityW = 0.1
            sparcityZ = 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 4 
porjection dim:	 3
Model size: 1.9936 KB
Accuracy for fold 0 with m = 4.000000,d = 3: 39 %
num prototypes:	 5 
porjection dim:	 3
Model size: 1.9816 KB
Accuracy for fold 0 with m = 5.000000,d = 3: 47 %
num prototypes:	 46 
porjection dim:	 2
Model size: 1.9904000000000002 KB
Accuracy for fold 0 with m = 46.000000,d = 2: 50 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 4 
porjection dim:	 3
Model size: 1.9936 KB
Accuracy for fold 1 with m = 4.000000,d = 3: 37 %
num prototypes:	 5 
porjection dim:	 3
Model size: 1.9816 KB
Accuracy for fold 1 with m = 5.000000,d = 3: 47 %
num prototypes:	 46 
porjection dim:	 2
Model size: 1.9904000000000002 KB
Accuracy for fold 1 with m = 46.000000,d = 2: 58 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 4 
porjection dim:	 3
Model size: 1.9936 KB
Accuracy for fold 2 with m

In [5]:
# FOR 4 KB
size = [4]
size_parameters = [[5, 14], [6, 7], [3, 8]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.5
        if (index_size == 1):
            sparcityZ = 0.1
        if (index_size == 2):
            sparcityW = 0.2
            sparcityZ = 0.2
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 14 
porjection dim:	 5
Model size: 3.976 KB
Accuracy for fold 0 with m = 14.000000,d = 5: 83 %
num prototypes:	 7 
porjection dim:	 6
Model size: 3.9872 KB
Accuracy for fold 0 with m = 7.000000,d = 6: 56 %
num prototypes:	 8 
porjection dim:	 3
Model size: 3.9872 KB
Accuracy for fold 0 with m = 8.000000,d = 3: 68 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 14 
porjection dim:	 5
Model size: 3.976 KB
Accuracy for fold 1 with m = 14.000000,d = 5: 82 %
num prototypes:	 7 
porjection dim:	 6
Model size: 3.9872 KB
Accuracy for fold 1 with m = 7.000000,d = 6: 64 %
num prototypes:	 8 
porjection dim:	 3
Model size: 3.9872 KB
Accuracy for fold 1 with m = 8.000000,d = 3: 69 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 14 
porjection dim:	 5
Model size: 3.976 KB
Accuracy for fold 2 with m = 14.000000,d = 5: 85 %
n

In [7]:
# FOR SMALL SIZE - sparsity on all parameters:1
size = [4, 8]

# For fold results
test_case = len(size)
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, size_limit in enumerate(size):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = math.trunc(utils.max_projection_dimensions(size[index_size], numOutputLabels, input_dimension, numOutputLabels))
        # numPrototypes: HYPERPARAMETR
        numPrototypes = math.trunc(utils.max_number_prototypes(size[index_size], projection_dimension, input_dimension, numOutputLabels))
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 19 
porjection dim:	 1
Model size: 3.972 KB
Accuracy for fold 0 with m = 19.000000,d = 1: 49 %
num prototypes:	 36 
porjection dim:	 2
Model size: 8.0 KB
Accuracy for fold 0 with m = 36.000000,d = 2: 71 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 19 
porjection dim:	 1
Model size: 3.972 KB
Accuracy for fold 1 with m = 19.000000,d = 1: 43 %
num prototypes:	 36 
porjection dim:	 2
Model size: 8.0 KB
Accuracy for fold 1 with m = 36.000000,d = 2: 71 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 19 
porjection dim:	 1
Model size: 3.972 KB
Accuracy for fold 2 with m = 19.000000,d = 1: 44 %
num prototypes:	 36 
porjection dim:	 2
Model size: 8.0 KB
Accuracy for fold 2 with m = 36.000000,d = 2: 68 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 19 
porjection dim:	 1
Mo

In [10]:
# FOR 8 KB
size = [8]
size_parameters = [[10, 21], [10, 30], [6, 11]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z: HAS TO BE DEFINED, CONSTRAINTS OF MEMORY
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.5
        if (index_size == 1):
            sparcityZ = 0.2
        if (index_size == 2):
            sparcityW = 0.2
            sparcityZ = 0.2
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 21 
porjection dim:	 10
Model size: 7.952 KB
Accuracy for fold 0 with m = 21.000000,d = 10: 87 %
num prototypes:	 30 
porjection dim:	 10
Model size: 7.952 KB
Accuracy for fold 0 with m = 30.000000,d = 10: 86 %
num prototypes:	 11 
porjection dim:	 6
Model size: 7.9664 KB
Accuracy for fold 0 with m = 11.000000,d = 6: 89 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 21 
porjection dim:	 10
Model size: 7.952 KB
Accuracy for fold 1 with m = 21.000000,d = 10: 89 %
num prototypes:	 30 
porjection dim:	 10
Model size: 7.952 KB
Accuracy for fold 1 with m = 30.000000,d = 10: 83 %
num prototypes:	 11 
porjection dim:	 6
Model size: 7.9664 KB
Accuracy for fold 1 with m = 11.000000,d = 6: 89 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 21 
porjection dim:	 10
Model size: 7.952 KB
Accuracy for fold 2 with m = 21.00000

In [11]:
# FOR 16 KB
size = [16]
size_parameters = [[15, 65], [10, 43]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nprojection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.5
        if (index_size == 1):
            sparcityW = 0.2
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 65 
porjection dim:	 15
Model size: 15.908 KB
Accuracy for fold 0 with m = 65.000000,d = 15: 90 %
num prototypes:	 43 
porjection dim:	 10
Model size: 15.984 KB
Accuracy for fold 0 with m = 43.000000,d = 10: 90 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 65 
porjection dim:	 15
Model size: 15.908 KB
Accuracy for fold 1 with m = 65.000000,d = 15: 91 %
num prototypes:	 43 
porjection dim:	 10
Model size: 15.984 KB
Accuracy for fold 1 with m = 43.000000,d = 10: 90 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 65 
porjection dim:	 15
Model size: 15.908 KB
Accuracy for fold 2 with m = 65.000000,d = 15: 90 %
num prototypes:	 43 
porjection dim:	 10
Model size: 15.984 KB
Accuracy for fold 2 with m = 43.000000,d = 10: 91 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 6

In [12]:
# FOR 16 KB - sparsity on all parameters:1
size = [16]
size_parameters = [[5, 5], [4, 30], [4, 61]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nprojection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 5 
projection dim:	 5
Model size: 15.98 KB
Accuracy for fold 0 with m = 5.000000,d = 5: 49 %
num prototypes:	 30 
projection dim:	 4
Model size: 14.224 KB
Accuracy for fold 0 with m = 30.000000,d = 4: 86 %
num prototypes:	 61 
projection dim:	 4
Model size: 15.96 KB
Accuracy for fold 0 with m = 61.000000,d = 4: 84 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 5 
projection dim:	 5
Model size: 15.98 KB
Accuracy for fold 1 with m = 5.000000,d = 5: 50 %
num prototypes:	 30 
projection dim:	 4
Model size: 14.224 KB
Accuracy for fold 1 with m = 30.000000,d = 4: 86 %
num prototypes:	 61 
projection dim:	 4
Model size: 15.96 KB
Accuracy for fold 1 with m = 61.000000,d = 4: 85 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 5 
projection dim:	 5
Model size: 15.98 KB
Accuracy for fold 2 with m = 5.000000,d = 5: 50 %
n

In [13]:
# FOR 32 KB
size = [32]
size_parameters = [[15, 37], [15, 41], [15, 55]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nprojection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z: HAS TO BE DEFINED, CONSTRAINTS OF MEMORY
        sparcityW, sparcityB, sparcityZ = 0.3, 1, 0.5
        if (index_size == 1):
            sparcityZ = 0.4
        if (index_size == 2):
            sparcityZ = 0.1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 37 
projection dim:	 15
Model size: 31.924 KB
Accuracy for fold 0 with m = 37.000000,d = 15: 91 %
num prototypes:	 41 
projection dim:	 15
Model size: 31.996 KB
Accuracy for fold 0 with m = 41.000000,d = 15: 92 %
num prototypes:	 55 
projection dim:	 15
Model size: 31.964 KB
Accuracy for fold 0 with m = 55.000000,d = 15: 89 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 37 
projection dim:	 15
Model size: 31.924 KB
Accuracy for fold 1 with m = 37.000000,d = 15: 92 %
num prototypes:	 41 
projection dim:	 15
Model size: 31.996 KB
Accuracy for fold 1 with m = 41.000000,d = 15: 91 %
num prototypes:	 55 
projection dim:	 15
Model size: 31.964 KB
Accuracy for fold 1 with m = 55.000000,d = 15: 90 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 37 
projection dim:	 15
Model size: 31.924 KB
Accuracy for fold 2 with m =

In [15]:
# FOR 32 KB - sparsity on all parameters:1
size = [32]
size_parameters = [[31, 130], [15, 182], [15, 130], [10, 208]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nprojection dim:\t", projection_dimension)
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 130 
projection dim:	 31
Model size: 118.536 KB
Accuracy for fold 0 with m = 130.000000,d = 31: 94 %
num prototypes:	 182 
projection dim:	 15
Model size: 65.24 KB
Accuracy for fold 0 with m = 182.000000,d = 15: 93 %
num prototypes:	 130 
projection dim:	 15
Model size: 60.04 KB
Accuracy for fold 0 with m = 130.000000,d = 15: 93 %
num prototypes:	 208 
projection dim:	 10
Model size: 48.0 KB
Accuracy for fold 0 with m = 208.000000,d = 10: 92 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 130 
projection dim:	 31
Model size: 118.536 KB
Accuracy for fold 1 with m = 130.000000,d = 31: 94 %
num prototypes:	 182 
projection dim:	 15
Model size: 65.24 KB
Accuracy for fold 1 with m = 182.000000,d = 15: 92 %
num prototypes:	 130 
projection dim:	 15
Model size: 60.04 KB
Accuracy for fold 1 with m = 130.000000,d = 15: 92 %
num prototypes:	 208 
projection dim:	 

In [16]:
# FOR 64 KB - sparsity on all parameters:1
size = [64]
size_parameters = [[15, 40], [15, 50], [15,  169]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nprojection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 40 
projection dim:	 15
Model size: 51.04 KB
Accuracy for fold 0 with m = 40.000000,d = 15: 92 %
num prototypes:	 50 
projection dim:	 15
Model size: 52.04 KB
Accuracy for fold 0 with m = 50.000000,d = 15: 92 %
num prototypes:	 169 
projection dim:	 15
Model size: 63.94 KB
Accuracy for fold 0 with m = 169.000000,d = 15: 93 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 40 
projection dim:	 15
Model size: 51.04 KB
Accuracy for fold 1 with m = 40.000000,d = 15: 92 %
num prototypes:	 50 
projection dim:	 15
Model size: 52.04 KB
Accuracy for fold 1 with m = 50.000000,d = 15: 92 %
num prototypes:	 169 
projection dim:	 15
Model size: 63.94 KB
Accuracy for fold 1 with m = 169.000000,d = 15: 92 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 40 
projection dim:	 15
Model size: 51.04 KB
Accuracy for fold 2 with m = 40

In [17]:
# Start print
knn = KNeighborsClassifier(n_neighbors = 5)
knn_results = {}
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # K-NN
    # test KNN
    knn.fit(train_inputs, train_targets)
    knn_foldscore = knn.score(test_inputs,test_targets)
    knn_results[fold] = knn_foldscore*100
    print("KNN accuracy:",knn_foldscore)
    print('--------------------------------')

sum = 0.0
for key, value in knn_results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average KNN: {sum/len(knn_results.items())} %')

--------------------------------
FOLD 0
--------------------------------
KNN accuracy: 0.943
--------------------------------
FOLD 1
--------------------------------
KNN accuracy: 0.9368333333333333
--------------------------------
FOLD 2
--------------------------------
KNN accuracy: 0.9406666666666667
--------------------------------
FOLD 3
--------------------------------
KNN accuracy: 0.9394166666666667
--------------------------------
FOLD 4
--------------------------------
KNN accuracy: 0.938
--------------------------------
Fold 0: 94.3 %
Fold 1: 93.68333333333332 %
Fold 2: 94.06666666666666 %
Fold 3: 93.94166666666666 %
Fold 4: 93.8 %
Average KNN: 93.95833333333333 %


In [18]:
# Start print
knn = KNeighborsClassifier(n_neighbors = 3)
knn_results = {}
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # K-NN
    # test KNN
    knn.fit(train_inputs, train_targets)
    knn_foldscore = knn.score(test_inputs,test_targets)
    knn_results[fold] = knn_foldscore*100
    print("KNN accuracy:",knn_foldscore)
    print('--------------------------------')

sum = 0.0
for key, value in knn_results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average KNN: {sum/len(knn_results.items())} %')

--------------------------------
FOLD 0
--------------------------------
KNN accuracy: 0.94525
--------------------------------
FOLD 1
--------------------------------
KNN accuracy: 0.941
--------------------------------
FOLD 2
--------------------------------
KNN accuracy: 0.946
--------------------------------
FOLD 3
--------------------------------
KNN accuracy: 0.9413333333333334
--------------------------------
FOLD 4
--------------------------------
KNN accuracy: 0.9420833333333334
--------------------------------
Fold 0: 94.525 %
Fold 1: 94.1 %
Fold 2: 94.6 %
Fold 3: 94.13333333333334 %
Fold 4: 94.20833333333334 %
Average KNN: 94.31333333333335 %


In [ ]:
import winsound
# to notify that I'm done running
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)
winsound.Beep(500, duration)